In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp

In [59]:
df = pd.read_excel('data.xlsx', header=None)
requirements = df.values.flatten()
requirements

array([ 11,   5,   4,   7,  16,   6,   5,   7,  13,   6,   5,   7,  12,
         5,   4,   6,   9,   5,   5,  11,  29,  21,  17,  20,  27,  13,
         9,  10,  16,   6,   5,   7,  11,   5,   5,   6,  12,   7,   7,
        10,  15,  10,   9,  11,  15,  10,  10,  16,  26,  21,  23,  36,
        50,  45,  45,  49,  57,  43,  40,  44,  52,  43,  42,  45,  52,
        41,  39,  41,  48,  35,  34,  35,  42,  34,  36,  43,  55,  48,
        54,  65,  80,  70,  74,  85, 101,  89,  88,  90, 100,  87,  88,
        89, 104,  89,  89,  90, 106,  96,  94,  99, 109,  99,  96, 102],
      dtype=int64)

In [60]:
total_week = 104
waste_rate = 0.2
k = 10
requirements = requirements[0:total_week]

In [61]:
hand_new = cp.Variable(total_week, integer=True)
hand_work = cp.Variable(total_week, integer=True)
hand_rest = cp.Variable(total_week, integer=True)
hand_teach = cp.Variable(total_week, integer=True)
hand_skill = cp.Variable(total_week, integer=True)
# body_new = cp.Variable(total_week, integer=True)
# body_work = cp.Variable(total_week, integer=True)
# body_rest = cp.Variable(total_week, integer=True)
# body_tested = cp.Variable(total_week, integer=True)
waste = cp.Variable(total_week, integer=True)

In [62]:
cons = [
    hand_work == 4 * requirements,
    # body_work == requirements,
    k * hand_teach >= hand_new,
    hand_skill == hand_work + hand_rest + hand_teach,
    # body_tested == body_work + body_rest,
    waste <= waste_rate * hand_work + 0.5,
    waste >= waste_rate * hand_work - 0.49999999,

    hand_new >= 0,
    hand_rest >= 0,
    hand_work >= 0,
    hand_teach >= 0,
    # body_new >= 0,
    # body_rest >= 0,
    # body_work >= 0,
    #waste >= 0,

    hand_skill[0] == 50,
    # body_tested[0] == 13
]

In [63]:
for t in range(total_week - 1):
    cons.append(hand_rest[t+1] >= hand_work[t] - waste[t])
    cons.append(hand_skill[t+1] == hand_skill[t] + hand_new[t] - waste[t])
    # cons.append(body_tested[t+1] == body_tested[t] + body_new[t])

In [64]:
obj = cp.Minimize(
    # 200 * cp.sum(body_new) + 
    100 * cp.sum(hand_new) + 
    5 * cp.sum(hand_rest) + 
    # 10 * cp.sum(body_rest) +
    10 * cp.sum(hand_new + hand_teach)
)

In [65]:
prob = cp.Problem(obj, cons)
prob.solve()

inf

In [66]:
hand_work.value[0:8], waste.value[0:8]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
hand_work.value, waste.value

(array([ 44.,  20.,  16.,  28.,  64.,  24.,  20.,  28.,  52.,  24.,  20.,
         28.,  48.,  20.,  16.,  24.,  36.,  20.,  20.,  44., 116.,  84.,
         68.,  80., 108.,  52.,  36.,  40.,  64.,  24.,  20.,  28.,  44.,
         20.,  20.,  24.,  48.,  28.,  28.,  40.,  60.,  40.,  36.,  44.,
         60.,  40.,  40.,  64., 104.,  84.,  92., 144., 200., 180., 180.,
        196., 228., 172., 160., 176., 208., 172., 168., 180., 208., 164.,
        156., 164., 192., 140., 136., 140., 168., 136., 144., 172., 220.,
        192., 216., 260., 320., 280., 296., 340., 404., 356., 352., 360.,
        400., 348., 352., 356., 416., 356., 356., 360., 424., 384., 376.,
        396., 436., 396., 384., 408.]),
 array([ 9.,  4.,  3.,  6., 13.,  5.,  4.,  6., 10.,  5.,  4.,  6., 10.,
         4.,  3.,  5.,  7.,  4.,  4.,  9., 23., 17., 14., 16., 22., 10.,
         7.,  8., 13.,  5.,  4.,  6.,  9.,  4.,  4.,  5., 10.,  6.,  6.,
         8., 12.,  8.,  7.,  9., 12.,  8.,  8., 13., 21., 17., 18., 29.,
  

In [ ]:
body_work.value

In [ ]:
hand_new.value

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,  42.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   1.,   0.,  12.,  35.,   7.,   0.,   3.,  29.,
         4.,   0.,  31.,  69.,  31.,  17.,  80., 120.,  61.,  23.,  54.,
        78.,  13.,   0.,  24.,  75.,  29.,   6.,  44.,  69.,  18.,   0.,
        30.,  61.,   8.,   0.,  16.,  56.,  23.,  15.,  67., 100.,  53.,
        46., 109., 140.,  70.,  47., 120., 159.,  80.,  33.,  79., 112.,
        58.,  37.,  82., 128.,  67.,  29.,  80., 136.,  90.,  42.,  92.,
       130.,  75.,  40.,  82.,   0.])